In this notebook:
1. We load finite element results files.
1. We split data in train/validation/test.
1. We compute some statistic.
1. We save all in one master file.

### General Libraries

In [1]:
import numpy as np
import h5py
from pathlib import Path

In [ ]:

# ========================= CONFIG =========================
DATA_DIR = Path("./data/Run1")
MASTER_FILE = Path("./master_data/rve_run1.h5")

N_RVE = 1000


In [ ]:

# Get list of .h5 files in the data directory
h5_files = sorted(list(DATA_DIR.glob("*.h5")))

N_RVE = len(h5_files)            # Total number of RVE files available

# Take dimensions and number of steps from the first file
first_file = h5_files[0]
with h5py.File(first_file, 'r') as f:
    # Read metadata attributes for dimensions and steps
    H = int(f.attrs['H_gp'])
    W = int(f.attrs['W_gp'])
    N_STEPS_PER_RVE = int(f.attrs['n_steps'])  # Time steps to use from each RVE

In [ ]:

import random
from sklearn.model_selection import train_test_split

# ====================== TRAIN / VALIDATION / TEST ======================

rnd_seed = 42

# Define fractions for train/validation/test
val_size = 0.20
test_size = 0.20
train_size = 1 - (val_size + test_size)

# Generate sets
rve_indexs = list(range(1, N_RVE + 1))
train_val_rves, test_rves = train_test_split(rve_indexs,
                                                   test_size = test_size,
                                                   random_state = rnd_seed)

train_rves, val_rves = train_test_split(train_val_rves,
                                            test_size=val_size / (train_size + val_size),
                                            random_state = rnd_seed)

# Compute amount of elements per set
ntrain = len(train_rves)
nval = len(val_rves)
ntest = len(test_rves)

print(f"Train RVEs: {ntrain} (f={train_size:.2f}) | Val: {nval}  (f={val_size:.2f}) | Test: {ntest}  (f={test_size:.2f})")
print(f"Total samples: {N_RVE * N_STEPS_PER_RVE}")


In [ ]:
from tqdm import tqdm

# ====================== CREATE MASTER HDF5 ======================

"""
Create Master HDF5 dataset for Dual-Encoder FNO
- x_local  : phase field (0=soft, 1=hard)
- x_global : [exx, eyy, gxy, E_soft, nu_soft, E_hard, nu_hard]
- y_local  : [Sxx, Syy, Sxy]
"""

with h5py.File(MASTER_FILE, 'w') as f:
    for split_name, rve_list in [("train", train_rves), 
                                 ("val",   val_rves), 
                                 ("test",  test_rves)]:
        
        N_split = len(rve_list) * N_STEPS_PER_RVE
        g = f.create_group(split_name)
        
        g.create_dataset('x_local',  shape=(N_split, H, W, 1), dtype=np.float32, compression='gzip')
        g.create_dataset('x_global', shape=(N_split, 7), dtype=np.float32, compression='gzip')
        g.create_dataset('y_local',  shape=(N_split, H, W, 3), dtype=np.float32, compression='gzip')
        
        idx = 0
        print(f"\nProcessing {split_name} split ({N_split} samples)...")
        
        for rve_id in tqdm(rve_list):
            file_path = DATA_DIR / f"rve_{rve_id:04d}.h5"
            
            with h5py.File(file_path, 'r') as src:
                fields = src['fields'][:]      # (N_steps, H, W, 4)
                macro  = src['macro'][:]       # (N_steps, 7)
                
                for step in range(min(N_STEPS_PER_RVE, len(fields))):
                    # x_local = phase field
                    g['x_local'][idx, :, :, 0] = fields[step, :, :, 0]
                    
                    # x_global
                    g['x_global'][idx] = macro[step]
                    
                    # y_local = stress field
                    g['y_local'][idx] = fields[step, :, :, 1:4]
                    
                    idx += 1

print(f"\n✅ Master HDF5 created successfully:")
print(f"   → {MASTER_FILE}")
print(f"   Train: {len(train_rves)*N_STEPS_PER_RVE} samples")
print(f"   Val:   {len(val_rves)*N_STEPS_PER_RVE} samples")
print(f"   Test:  {len(test_rves)*N_STEPS_PER_RVE} samples")


In [ ]:
# ====================== COMPUTE AND SAVE STATS ======================

print("\nComputing normalization statistics (from train only)...")

with h5py.File(MASTER_FILE, 'a') as f:
    train = f['train']
    
    x_local  = train['x_local'][:]
    x_global = train['x_global'][:]
    y_local  = train['y_local'][:]
    
    stats = f.create_group('stats')
    
    stats.create_dataset('mean_x_local',  data=x_local.mean(axis=(0,1,2), keepdims=True).squeeze(0))
    stats.create_dataset('std_x_local',   data=x_local.std(axis=(0,1,2), keepdims=True).squeeze(0) + 1e-8)
    
    stats.create_dataset('mean_x_global', data=x_global.mean(axis=0, keepdims=True).squeeze(0))
    stats.create_dataset('std_x_global',  data=x_global.std(axis=0, keepdims=True).squeeze(0) + 1e-8)
    
    stats.create_dataset('mean_y_local',  data=y_local.mean(axis=(0,1,2), keepdims=True).squeeze(0))
    stats.create_dataset('std_y_local',   data=y_local.std(axis=(0,1,2), keepdims=True).squeeze(0) + 1e-8)

print("   Statistics saved in group '/stats'")
print("\n✅ Master dataset is ready for training!")


Computing normalization statistics (from train only)...


ValueError: Unable to synchronously create group (name already exists)